# Blackjack Solver
Shantanu Laghate, November 2024

## Problem Definition
Objective of the game:
- obtain cards the sum of whose numerican values is as great as possible without exceeding 21.
- All face cards count as 10, ace can count as either 1 or 11.
- Player competes independently against dealer.

Gameplay:
- Game begins with two cards dealt to both dealer and player. One of dealer's cards is face up, other is face down.
- If player has 21 immediately (Ace + 10), this is a natural and player wins unless dealer also has a natural in which case it's a draw
- Otherwise, player has two actions:
    + Hit (request additional card)
        * If the player exceeds 21 (goes bust), he loses
    + Stick (stop with current cards)
        * After stick, it becomes the dealer's turn.
- The dealer follows a fixed strategy without choice: stick on any sum 17 or greater, and hits otherwise. If dealer goes bust, player wins. 
- Final outcome decided by whose final sum is closer to 21.

Assumption: cards are dealth from an infinite deck (with replacement), so that there is no advantage to keeping track of the cards already dealt. 

## Formulation as MDP
We can formulate Blackjack as an episodic finite MDP. Each hand of Blackjack is an episode. 
- Rewards: +1, -1, 0 are given for winning, losing, and drawing. All rewards within a game are 0.
- Actions: Hit or Stick
- State: Three variables (Usable Ace, Player's cards and dealer's showing card)
    + Usable ace: if player has ace that can be used as 11 without going bust.
    + Current Sum (12-21) - since if the sum is 11 or lower the player will always hit, there is no decision to make. 
    + Dealer's one showing card (ace-10)
    + Total 200 states


## On-policy Monte Carlo Algorithm with Exploring Starts 
With a Monte Carlo method, we will simulate millions of hands of blackjack to find the optimal policy. We will estimate the Action-Value function, which is the expected return from a given state $s$, if the immediate following action is $a$. This estimate is given by $Q(s, a)$.

Here is the pseudocode for the algorithm:
```
Initialize:
    pi[s], arbitrarily for all s in S, a in A
    Q[s, a] = 0 for all s in S, a in A
    Count[s, a] = 0 for all s in S, a in A

Loop infinitely (for each episode):
    Choose S_0, A_0 randomly such that all pairs have probability > 0
    Generate an episode from S_0, A_0 following pi: S_0, A_0, R_1, ..., S_T-1, A_T-1, R_T
    G = 0
    Loop for each step in the episode t = T-1, T-2, ..., 0:
    G = gamma*G + R_{t+1}
    Unless the pair (St, At) appears earlier in the episode:
        Count[St, At] += 1
        Q(St, At) = Q[St, At] + 1/Count[St, At]*(G - Q[St, At]) # incremental update algorithm
        pi[St] = argmax_a Q[St, a]
```

## Representing State and Action Spaces, Policy
Since there is no decision to be made when `current_sum < 12`, the indexing will correspond to `current_sum-12`.
State is a `(usable_ace, current_sum, dealer_card)` tuple. Action is an enum between `HIT = 0 , STICK = 1`. The action-value function can then be represented by a 4d numpy array: `Q[usable_ace][current_sum][dealer_card][action]`. 

The policy can be represented as a 3d numpy array: `pi[usable_ace, current_sum][dealer_card] in {HIT, STICK}`. 

## Performing the Iteration 
To perform exploring starts, we first select a random state tuple, and random action for our S0, A0. Follow the policy `pi` until the player either goes bust or chooses `STICK`. If the player is not bust, iterate on the dealer until dealer sum is 17 or higher (or dealer goes bust). Finally, compare sums and delegate reward accordingly. All rewards until the last reward will be 0 in the episode. We will keep track of states, actions, and rewards for the entire episode as they are generated. The dealer iteration does not need to be kept track, only the final result is captured in the last reward. 




In [2]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from typing import List, Tuple, Optional
import logging

logging.basicConfig(
    level=logging.DEBUG,  # Set default logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [3]:
class Action(Enum):
    HIT = 0
    STICK = 1

SUM_OFFSET = 12

In [9]:
def random_card_value() -> Tuple[int, bool]:
    # cards 1, 14, 27, 40 are aces
    card = np.random.randint(1, 53)

    rank = card % 13
    is_ace = rank == 1
    
    if is_ace:
        value = 11
    elif rank >= 10:
        value = 10
    else:
        value = rank

    logger.debug(f"Picked card: {value}, {is_ace}")
    return (value, is_ace)
    
def next_state(usable_ace, current_sum, next_card: Tuple[int, bool]) -> Optional[Tuple[int, int]]:
    """
    Given the current state and next card, return next (usable_ace, current_sum) or return None if player went bust
    """
    value, is_ace = next_card
    logger.debug(f"Old state: {usable_ace}, {current_sum}, Next card: {next_card}")

    next_sum = current_sum + value
    next_usable_ace = usable_ace or is_ace
    if next_sum > 21:
        if (usable_ace or is_ace):
            # if you already have a usable ace and you drew one, you still have one left
            # if you only just drew one, you have none
            # this is a XNOR operation
            next_usable_ace = not (usable_ace ^ is_ace)
            next_sum -= 10
        else:
            # went bust
            logger.debug("Went Bust")
            return None
    logger.debug(f"Transitioned from state {usable_ace}, {current_sum} to {next_usable_ace}, {next_sum}")
    return (next_usable_ace, next_sum)
        

def player_draw_until_stick_or_bust(usable_ace, start_sum, dealer_card = None, pi = None) -> Tuple[bool, List[Tuple[bool, int]]]:
    """
    Follow policy pi until stick or bust
    Can be used by both player or dealer
        For player usage, provide dealer_card and policy pi
        For dealer usage, leave both as None
    Note that dealer_card is only used in this process to select action for the player based on pi
    Output: (end_stick, [(usable_ace, current_sum)])
    """
    if pi:
        logger.debug("PLAYER mode")
    else:
        logger.debug("DEALER mode")
        
    state_sequence = [(usable_ace, start_sum)]
    while True:
        cur_usable_ace, cur_sum = state_sequence[-1]
        logger.debug(f"Current step: {cur_usable_ace}, {cur_sum}")
        if pi:
            # player
            action = pi[cur_usable_ace, cur_sum-SUM_OFFSET, dealer_card]
        else:
            # dealer
            if cur_sum >= 17:
                
                action = Action.STICK
            else:
                action = Action.HIT

        if action == Action.STICK:
            logger.debug("Sticking!")
            return (True, state_sequence)
        
        if action == Action.HIT:
            next_card = random_card_value()
            next_step = next_state(cur_usable_ace, cur_sum, next_card)
            if not next_step:
                # episode ends here
                logger.debug("Went Bust!")
                return (False, state_sequence)
            else:
                state_sequence.append(next_step)


In [35]:
player_draw_until_stick_or_bust(False, 7)
#next_state(1, 17, (11, True))

2024-11-10 22:16:05,027 - __main__ - DEBUG - DEALER mode
2024-11-10 22:16:05,029 - __main__ - DEBUG - Current step: False, 7
2024-11-10 22:16:05,030 - __main__ - DEBUG - Picked card: 2, False
2024-11-10 22:16:05,031 - __main__ - DEBUG - Old state: False, 7, Next card: (2, False)
2024-11-10 22:16:05,032 - __main__ - DEBUG - Transitioned from state False, 7 to False, 9
2024-11-10 22:16:05,032 - __main__ - DEBUG - Current step: False, 9
2024-11-10 22:16:05,033 - __main__ - DEBUG - Picked card: 11, True
2024-11-10 22:16:05,033 - __main__ - DEBUG - Old state: False, 9, Next card: (11, True)
2024-11-10 22:16:05,034 - __main__ - DEBUG - Transitioned from state False, 9 to True, 20
2024-11-10 22:16:05,035 - __main__ - DEBUG - Current step: True, 20
2024-11-10 22:16:05,035 - __main__ - DEBUG - Sticking!


(True, [(False, 7), (False, 9), (True, 20)])

In [ ]:
type State = Tuple(int, int, int)
type Reward = int
type Episode = List[Tuple(State, Action, Reward)]


def generate_random_episode_ES(pi) -> Episode:
    # generate a random starting state and action
    usable_ace = np.random.randint(0, 2)
    current_sum = np.random.randint(0, 10)
    dealer_card = np.random.randint(0, 10)
    first_action = np.random.choice(list(Action))

    logger.debug(f"Starting State: {usable_ace}, {current_sum}, {dealer_card}, {first_action}")
    
    # PRE: last_state, last_action are relevant. 
    # POST: dealer either wins, 
                
            
        
    


def monte_carlo_blackjack(num_episodes=1):
    # initial policy = HIT in all states.
    pi = np.zeros((2, 10, 10))

    # starting action-value estimates 0 for all state-action pairs
    Q = np.zeros((2, 10, 10, 2))
    counts = np.zeros((2, 10, 10, 2))

    for i in range(num_episodes):
        episode = generate_random_episode_ES(pi)

        # keep track of first-visits to action-state pairs
        first_visits = {}
        for t, state_action_reward in enumerate(episode):
            state, action, _ = state_action_reward
            if (state, action) not in first_visits:
                first_visits[(state, action)] = t
        
        G = 0
        for t, state_action_reward in reversed(list(enumerate(episode))):
            state, action, reward = state_action_reward
            G = gamma*G + reward
            
            if t == first_visits[(state, action)]:    
                counts[state][action] += 1
                Q[state][action] = Q[state][action] + 1/counts[state][action]*(G - Q[state][action])
                pi[state] = np.argmax([Q[state][0], Q[state][1]])

    return pi, Q

In [ ]:
episode = [(1, 1, 1), (2, 2, 2)]
for t, sa in reversed(list(enumerate(episode))):
    a, b, c = sa
    print(t, a, b, c)

In [ ]:
*(1, 1, 1)

In [ ]:
np.argmax([0, 1])

In [ ]:
np.random.choice(list(Action))

In [ ]:
usable_ace = 0
is_ace = False
not (usable_ace^is_ace)